In [2]:
import pickle
import numpy as np
from mtranslate import translate
import gensim

with open('modelParameters.pickle', 'rb') as handle:
    modelParametersDict = pickle.load(handle)

Wxh = modelParametersDict['Wxh']
Whh = modelParametersDict['Whh']
Why = modelParametersDict['Why']
bh = modelParametersDict['bh']
by = modelParametersDict['by']
hprev = modelParametersDict['hprev']
char_en_to_vec = modelParametersDict['char_en_to_vec']
vocab_size = 100
modelDe = gensim.models.Word2Vec.load('german_word2vec_new')


test_sentence = ['<', 'he', 'took', 'the', 'bus', 'back', '>']
test_sentence_vec = [char_en_to_vec[ch] for ch in test_sentence]
test_sentence_de= translate(' '.join(test_sentence),'de', 'en').split()



def sample(h, n):
  n = len(test_sentence_vec)
  translated_vecs = []
  for t in range(n):
    #x = np.zeros((vocab_size, 1))
    #x[char_to_ix[test_sentence[t]]] = 1
    x = np.reshape(np.array(test_sentence_vec[t]), (vocab_size,1) )
        
    h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
    y = np.dot(Why, h) + by   
    translated_vecs.append(y)
  return translated_vecs


slen = len(test_sentence_vec)
sample_vecs = sample(hprev, slen)

print('---------------')
print('Input Sentence : '+' '.join(test_sentence))
print('Expected Translation by Google : '+' '.join(test_sentence_de))

i = 0
for v in sample_vecs:
    top_n = modelDe.similar_by_vector(v.flatten(), topn=3, restrict_vocab=None)
    print("word "+str(i+1)+" : ",end='')
    for j in range(3):
        print(top_n[j][0],"%.2f" %top_n[j][1] ," , " ,end=' ')
    i=i+1
    print()


---------------
Input Sentence : < he took the bus back >
Expected Translation by Google : <Er hat den Bus zurück gebracht
word 1 : < 1.00  ,  > 0.90  ,  ist 0.79  ,  
word 2 : er 0.98  ,  > 0.87  ,  < 0.80  ,  
word 3 : ist 0.81  ,  mir 0.54  ,  das 0.53  ,  
word 4 : die 0.87  ,  > 0.76  ,  < 0.75  ,  
word 5 : das 0.53  ,  ist 0.51  ,  sind 0.50  ,  
word 6 : < 0.70  ,  das 0.61  ,  ist 0.54  ,  
word 7 : > 1.00  ,  < 0.89  ,  ist 0.81  ,  
